# FITS File Editor

The purpose of this tutorial is to create and append a new column into an existing FITS file table.

In [1]:
import numpy as np 
import pandas as pd 
from astropy.table import Table

Read in all files as Astropy Table objects.

In [2]:
pd.set_option('display.max_columns', None)

filename_1 = "path_to_file/filename"
filename_2 = "path_to_file/filename"

catalog_matcha = Table.read(filename_1)
catalog_gold= Table.read(filename_2)

In [3]:
print(f"Number of rows:{len(catalog_matcha)}")
print(f"Number of columns:{len(catalog_matcha.colnames)}")
catalog_matcha[0:5]

Number of rows:1092
Number of columns:111


Name,MEM_MATCH_ID,Analyzed,Detected,redMaPPer_ra,redMaPPer_dec,Redshift,Obsids,lambda,lambda_err_low,lambda_err_high,band_lumin_upper_lim_undetected,band_lumin_upper_lim_detected,lumin_no_tx,lumin_no_tx_err_low,lumin_no_tx_err_high,x_ray_peak_ra,x_ray_peak_dec,500_kiloparsecs_ra,500_kiloparsecs_dec,500_kiloparsecs_temperature,500_kiloparsecs_temperature_err_low,500_kiloparsecs_temperature_err_high,500_kiloparsecs_band_lumin,500_kiloparsecs_band_lumin_err_low,500_kiloparsecs_band_lumin_err_high,500_kiloparsecs_bolo_lumin,500_kiloparsecs_bolo_lumin_err_low,500_kiloparsecs_bolo_lumin_err_high,500_kiloparsecs_radius,500_kiloparsecs_radius_err_low,500_kiloparsecs_radius_err_high,500_kiloparsecs_SNR,500_kiloparsecs_SNR_err_low,500_kiloparsecs_SNR_err_high,r2500_ra,r2500_dec,r2500_temperature,r2500_temperature_err_low,r2500_temperature_err_high,r2500_band_lumin,r2500_band_lumin_err_low,r2500_band_lumin_err_high,r2500_bolo_lumin,r2500_bolo_lumin_err_low,r2500_bolo_lumin_err_high,r2500_radius,r2500_radius_err_low,r2500_radius_err_high,r2500_SNR,r2500_SNR_err_low,r2500_SNR_err_high,r500_ra,r500_dec,r500_temperature,r500_temperature_err_low,r500_temperature_err_high,r500_band_lumin,r500_band_lumin_err_low,r500_band_lumin_err_high,r500_bolo_lumin,r500_bolo_lumin_err_low,r500_bolo_lumin_err_high,r500_radius,r500_radius_err_low,r500_radius_err_high,r500_SNR,r500_SNR_err_low,r500_SNR_err_high,r500_core_cropped_ra,r500_core_cropped_dec,r500_core_cropped_temperature,r500_core_cropped_temperature_err_low,r500_core_cropped_temperature_err_high,r500_core_cropped_band_lumin,r500_core_cropped_band_lumin_err_low,r500_core_cropped_band_lumin_err_high,r500_core_cropped_bolo_lumin,r500_core_cropped_bolo_lumin_err_low,r500_core_cropped_bolo_lumin_err_high,r500_core_cropped_radius,r500_core_cropped_radius_err_low,r500_core_cropped_radius_err_high,r500_core_cropped_SNR,r500_core_cropped_SNR_err_low,r500_core_cropped_SNR_err_high,peak_fix,masked,bad_mode,quick look bad,obsid_bad,merger,Serendipitous,edge_centroid,edge_peak,edge_r2500,edge_r500,edge_bkgd,overlap_r2500,overlap_r500,overlap_bkgd,notes,Group 1 Check,redmapper wrong,right RA,right DEC,right in top 5,right not member,ambiguous,Chandra Notes,Tx_in_xmm
bytes15,int64,bool,bool,float64,float64,float64,bytes96,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,bytes4,int64,int64,bytes4,bytes26,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,bytes106,bytes14,int64,float64,float64,int64,int64,int64,bytes145,int32
catalogue_1,2,True,True,83.231901970306,-37.0266894362176,0.2833377420902252,15112,199.432,5.87942,5.87942,nan,nan,4.5843e+44,4.815e+42,4.8632e+42,83.2325,-37.02649,83.23211,-37.02644,11.344,0.68483,0.68878,4.3399e+44,6.51e+42,6.5277e+42,1.7469e+45,2.866e+43,2.9201e+43,500.0,0.0,0.0,94.277,1.087,1.087,83.23188,-37.02643,11.166,0.63832,0.6424,4.944e+44,6.9038e+42,6.8134e+42,1.9782e+45,3.086e+43,3.0796e+43,672.57,23.048,22.571,101.61,1.093,1.093,83.23226,-37.02672,10.934,0.85751,0.85747,5.3675e+44,8.55e+42,8.5518e+42,2.1305e+45,4.5892e+43,4.6121e+43,1511.0,49.857,48.953,79.355,1.039,1.039,83.23226,-37.02672,12.746,1.8843,2.0198,2.7238e+44,6.0129e+42,8.2416e+42,1.1614e+45,6.8477e+43,2.9554e+43,1511.0,49.857,48.953,46.384,1.0189,1.0189,N/A,0,0,N/A,N/A,0,0,0,0,0,0,0,0,0,0,N/A,yes,0,9999.0,9999.0,0,0,0,N/A,1
catalogue_2,3,True,True,347.092550205901,-2.19213688831977,0.29250583052635193,"4962, 9372",163.583,4.29459,4.29459,nan

In [4]:
print(f"Number of rows:{len(catalog_gold)}")
print(f"Number of columns:{len(catalog_gold.colnames)}")
catalog_gold[0:5]

Number of rows:21092
Number of columns:69


MEM_MATCH_ID,RA,DEC,MODEL_MAG [4],MODEL_MAGERR [4],REFMAG,REFMAG_ERR,ZRED,ZRED_E,ZRED_CHISQ,BCG_SPEC_Z,Z_SPEC_INIT,Z_INIT,Z,LAMBDA_CHISQ,LAMBDA_CHISQ_E,LAMBDA_ZRED,LAMBDA_ZRED_E,R_LAMBDA,SCALEVAL,MASKFRAC,GEOMASKFRAC,C_LAMBDA [3],C_LAMBDA_ERR [3],MAG_LAMBDA_ERR [4],CHISQ,Z_LAMBDA,Z_LAMBDA_E,Z_LAMBDA_NITER,EBV_MEAN,LNLAMLIKE,LNBCGLIKE,LNLIKE,PZBINS [21],PZ [21],NCROSS,RMASK,RA_ORIG,DEC_ORIG,W,DLAMBDA_DZ,DLAMBDA_DZ2,DLAMBDAVAR_DZ,DLAMBDAVAR_DZ2,LAMBDA_CHISQ_C,LAMBDA_CHISQ_CE,NCENT,NCENT_GOOD,RA_CENT [5],DEC_CENT [5],ID_CENT [5],LAMBDA_CHISQ_CENT [5],ZLAMBDA_CENT [5],P_BCG [5],P_CEN [5],Q_CEN [5],P_FG [5],Q_MISS,P_SAT [5],P_C [5],BCG_ILUM,ILUM,Z_LAMBDA_RAW,Z_LAMBDA_E_RAW,LIM_EXPTIME,LIM_LIMMAG_DERED,LIM_LIMMAG,LIM_LIMMAG_HARD,MASKGAL_INDEX
int32,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,float32,float32,float32,float32,float32,float32,int16,float32,float64,float64,float32,float32,float32,float32,float32,float32,float32,int16,int16,float64,float64,int64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16
1,43.5645791817007,-58.9529590028976,20.05658 .. 17.211803,0.0061925324 .. 0.0032607391,17.211803,0.0032607391,0.41990262,0.019821867,2.0704932,-1.0,-1.0,0.4450967,0.4281561,221.67424,5.712534,192.14061,5.1811213,1.1725816,1.0148627,0.017677214,0.0,1.7302246 .. 0.33943343,0.0010073867 .. 0.00025456835,0.0008543046 .. 0.0,1.4507834,0.42608216,0.009337322,2,0.021020807,468.0588,-1.1185727,466.94025,0.37455446 .. 0.4776099,1.43873685e-05 .. 4.2987836e-06,1,2.7900147,43.5816101710759,-58.9506770104227,1.3288823,1.5503892,-104.31785,49.340786,27618.16,221.67424,0.0,5,1,43.5645791817007 .. 43.5569977737336,-58.9529590028976 .. -58.9390662130826,329763550 .. 329762991,221.67424 .. 0.0,0.42675602 .. 0.0,1.0 .. 1.0,0.998865 .. 1.1687765e-05,0.93537223 .. 1.0944834e-05,6.3997913e-07 .. 0.0030109354,-5.499822,0.0011343517 .. 0.9969774,0.93636745 .. 0.00017215396,6.752811,154.95233,0.42675602,0.0029298565,0.2809465,22.476273,22.476273,23.559084,93
2,83.231901970306,-37.0266894362176,17.967796 .. 15.711882,0.0015648057 .. 0.0010313771,15.711882,0.0010313771,0.25323626,0.0108621,1.8223227,-1.0,-1.0,0.25323626,0.2871882,199.4325,5.87942,184.4521,5.552468,1.1480458,1.0425289,0.04005748,0.0,1.4144971 .. 0.28641897,0.00017805232 .. 5.6211866e-05,0.00014226134 .. 0.0,1.1697916,0.28333774,0.005391381,1,0.039437305,421.90625,-1.0352563,420.871,0.25237316 .. 0.31430233,4.2654037e-06 .. 3.7891057e-06,1,2.6603746,83.231901970306,-37.0266894362176,1.5115601,3.2341394,221.73894,162.52467,58253.285,199.4325,0.0,5,1,83.231901970306 .. 83.2256237938038,-37.0266894362176 .. -37.0185740256312,434820709 .. 434820621,199.4325 .. 0.0,0.28375688 .. 0.0,1.0 .. 1.0,0.99960846 .. 1.4246054e-05,0.9990755 .. 1.4238458e-05,4.8695372e-05 .. 4.8423062e-05,-16.255661,0.00034284755 .. 0.99993736,0.9994666 .. 0.026009087,10.781266,160.15294,0.28375688,0.002188141,0.2276197,22.317131,22.317131,23.559084,64
3,347.092550205901,-2.19213688831977,18.024233 .. 15.605037,0.0026223608 .. 0.0015591898,15.605037,0.0015591898,0.2625042,0.011154525,3.97064,-1.0,-1.0,0.26911718,0.29409084,163.5833,4.2945933,158.96104,4.0634966,1.1034377,1.0123166,0.011719465,0.0,1.4740494 .. 0.31551334,0.00036889763 .. 0.00010674338,0.0002983139 .. 0.0,2.3309622,0.29250583,0.0057010963,1,0.09085343,395.27588,-0.21557358,395.0603,0.26078478 .. 0.32422686,2.10779e-05 .. 5.034007e-06,1,2.4334204,347.089282760688,-2.19067422831961,1.7688129,4.4487863,151.61554,170.70558,33972.55,163.52493,0.30298,5,2,347.092550205901 .. 347.099384487043,-2.19213688831977 .. -2.19344409486259,113660691 .. 113660879,163.5833 .. 163.23244,0.2932724 .. 0.0,1.0 .. 1.0,0.961471 .. 5.574278e-06,0.9608368 .. 5.570601e-06,0.001909672 .. 1.5593596e-05,-26.678312,0.0366193

Now that we have both tables, let's go ahead and create a new table called `catalog_cross1` made up of rows in common between the `MEM_MATCH_ID` column in both tables. We use `np.isin()` method to return an array of booleans which we then pass to the `catalog_matcha` such that we keep all of its columns and return a subset of rows. 

Thought process:
* Table1 & Table2
* Check if values in columnA from Table1 are in columnB from Table2.
* If yes, then keep that row in Table1. If no, don't keep.


In [5]:
catalog_cross1 = catalog_matcha[np.isin(catalog_matcha['MEM_MATCH_ID'], catalog_gold['MEM_MATCH_ID'])]

print(f'Dimensions of df_cross2 {len(catalog_cross1.colnames)} and {len(catalog_cross1)}.')
catalog_cross1[0:5]

Dimensions of df_cross2 111 and 501.


Name,MEM_MATCH_ID,Analyzed,Detected,redMaPPer_ra,redMaPPer_dec,Redshift,Obsids,lambda,lambda_err_low,lambda_err_high,band_lumin_upper_lim_undetected,band_lumin_upper_lim_detected,lumin_no_tx,lumin_no_tx_err_low,lumin_no_tx_err_high,x_ray_peak_ra,x_ray_peak_dec,500_kiloparsecs_ra,500_kiloparsecs_dec,500_kiloparsecs_temperature,500_kiloparsecs_temperature_err_low,500_kiloparsecs_temperature_err_high,500_kiloparsecs_band_lumin,500_kiloparsecs_band_lumin_err_low,500_kiloparsecs_band_lumin_err_high,500_kiloparsecs_bolo_lumin,500_kiloparsecs_bolo_lumin_err_low,500_kiloparsecs_bolo_lumin_err_high,500_kiloparsecs_radius,500_kiloparsecs_radius_err_low,500_kiloparsecs_radius_err_high,500_kiloparsecs_SNR,500_kiloparsecs_SNR_err_low,500_kiloparsecs_SNR_err_high,r2500_ra,r2500_dec,r2500_temperature,r2500_temperature_err_low,r2500_temperature_err_high,r2500_band_lumin,r2500_band_lumin_err_low,r2500_band_lumin_err_high,r2500_bolo_lumin,r2500_bolo_lumin_err_low,r2500_bolo_lumin_err_high,r2500_radius,r2500_radius_err_low,r2500_radius_err_high,r2500_SNR,r2500_SNR_err_low,r2500_SNR_err_high,r500_ra,r500_dec,r500_temperature,r500_temperature_err_low,r500_temperature_err_high,r500_band_lumin,r500_band_lumin_err_low,r500_band_lumin_err_high,r500_bolo_lumin,r500_bolo_lumin_err_low,r500_bolo_lumin_err_high,r500_radius,r500_radius_err_low,r500_radius_err_high,r500_SNR,r500_SNR_err_low,r500_SNR_err_high,r500_core_cropped_ra,r500_core_cropped_dec,r500_core_cropped_temperature,r500_core_cropped_temperature_err_low,r500_core_cropped_temperature_err_high,r500_core_cropped_band_lumin,r500_core_cropped_band_lumin_err_low,r500_core_cropped_band_lumin_err_high,r500_core_cropped_bolo_lumin,r500_core_cropped_bolo_lumin_err_low,r500_core_cropped_bolo_lumin_err_high,r500_core_cropped_radius,r500_core_cropped_radius_err_low,r500_core_cropped_radius_err_high,r500_core_cropped_SNR,r500_core_cropped_SNR_err_low,r500_core_cropped_SNR_err_high,peak_fix,masked,bad_mode,quick look bad,obsid_bad,merger,Serendipitous,edge_centroid,edge_peak,edge_r2500,edge_r500,edge_bkgd,overlap_r2500,overlap_r500,overlap_bkgd,notes,Group 1 Check,redmapper wrong,right RA,right DEC,right in top 5,right not member,ambiguous,Chandra Notes,Tx_in_xmm
bytes15,int64,bool,bool,float64,float64,float64,bytes96,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,bytes4,int64,int64,bytes4,bytes26,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,bytes106,bytes14,int64,float64,float64,int64,int64,int64,bytes145,int32
catalogue_1,2,True,True,83.231901970306,-37.0266894362176,0.2833377420902252,15112,199.432,5.87942,5.87942,nan,nan,4.5843e+44,4.815e+42,4.8632e+42,83.2325,-37.02649,83.23211,-37.02644,11.344,0.68483,0.68878,4.3399e+44,6.51e+42,6.5277e+42,1.7469e+45,2.866e+43,2.9201e+43,500.0,0.0,0.0,94.277,1.087,1.087,83.23188,-37.02643,11.166,0.63832,0.6424,4.944e+44,6.9038e+42,6.8134e+42,1.9782e+45,3.086e+43,3.0796e+43,672.57,23.048,22.571,101.61,1.093,1.093,83.23226,-37.02672,10.934,0.85751,0.85747,5.3675e+44,8.55e+42,8.5518e+42,2.1305e+45,4.5892e+43,4.6121e+43,1511.0,49.857,48.953,79.355,1.039,1.039,83.23226,-37.02672,12.746,1.8843,2.0198,2.7238e+44,6.0129e+42,8.2416e+42,1.1614e+45,6.8477e+43,2.9554e+43,1511.0,49.857,48.953,46.384,1.0189,1.0189,N/A,0,0,N/A,N/A,0,0,0,0,0,0,0,0,0,0,N/A,yes,0,9999.0,9999.0,0,0,0,N/A,1
catalogue_2,3,True,True,347.092550205901,-2.19213688831977,0.29250583052635193,"4962, 9372",163.583,4.29459,4.29459,nan

Similar process as above is applied here in reverse.

In [6]:
catalog_cross2 = catalog_gold[np.isin(catalog_gold['MEM_MATCH_ID'], catalog_matcha['MEM_MATCH_ID'])]

print(f'Dimensions of df_cross2 {len(catalog_cross2.colnames)} and {len(catalog_cross2)}.')
catalog_cross2[0:5]

Dimensions of df_cross2 69 and 501.


MEM_MATCH_ID,RA,DEC,MODEL_MAG [4],MODEL_MAGERR [4],REFMAG,REFMAG_ERR,ZRED,ZRED_E,ZRED_CHISQ,BCG_SPEC_Z,Z_SPEC_INIT,Z_INIT,Z,LAMBDA_CHISQ,LAMBDA_CHISQ_E,LAMBDA_ZRED,LAMBDA_ZRED_E,R_LAMBDA,SCALEVAL,MASKFRAC,GEOMASKFRAC,C_LAMBDA [3],C_LAMBDA_ERR [3],MAG_LAMBDA_ERR [4],CHISQ,Z_LAMBDA,Z_LAMBDA_E,Z_LAMBDA_NITER,EBV_MEAN,LNLAMLIKE,LNBCGLIKE,LNLIKE,PZBINS [21],PZ [21],NCROSS,RMASK,RA_ORIG,DEC_ORIG,W,DLAMBDA_DZ,DLAMBDA_DZ2,DLAMBDAVAR_DZ,DLAMBDAVAR_DZ2,LAMBDA_CHISQ_C,LAMBDA_CHISQ_CE,NCENT,NCENT_GOOD,RA_CENT [5],DEC_CENT [5],ID_CENT [5],LAMBDA_CHISQ_CENT [5],ZLAMBDA_CENT [5],P_BCG [5],P_CEN [5],Q_CEN [5],P_FG [5],Q_MISS,P_SAT [5],P_C [5],BCG_ILUM,ILUM,Z_LAMBDA_RAW,Z_LAMBDA_E_RAW,LIM_EXPTIME,LIM_LIMMAG_DERED,LIM_LIMMAG,LIM_LIMMAG_HARD,MASKGAL_INDEX
int32,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,float32,float32,float32,float32,float32,float32,int16,float32,float64,float64,float32,float32,float32,float32,float32,float32,float32,int16,int16,float64,float64,int64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16
2,83.231901970306,-37.0266894362176,17.967796 .. 15.711882,0.0015648057 .. 0.0010313771,15.711882,0.0010313771,0.25323626,0.0108621,1.8223227,-1.0,-1.0,0.25323626,0.2871882,199.4325,5.87942,184.4521,5.552468,1.1480458,1.0425289,0.04005748,0.0,1.4144971 .. 0.28641897,0.00017805232 .. 5.6211866e-05,0.00014226134 .. 0.0,1.1697916,0.28333774,0.005391381,1,0.039437305,421.90625,-1.0352563,420.871,0.25237316 .. 0.31430233,4.2654037e-06 .. 3.7891057e-06,1,2.6603746,83.231901970306,-37.0266894362176,1.5115601,3.2341394,221.73894,162.52467,58253.285,199.4325,0.0,5,1,83.231901970306 .. 83.2256237938038,-37.0266894362176 .. -37.0185740256312,434820709 .. 434820621,199.4325 .. 0.0,0.28375688 .. 0.0,1.0 .. 1.0,0.99960846 .. 1.4246054e-05,0.9990755 .. 1.4238458e-05,4.8695372e-05 .. 4.8423062e-05,-16.255661,0.00034284755 .. 0.99993736,0.9994666 .. 0.026009087,10.781266,160.15294,0.28375688,0.002188141,0.2276197,22.317131,22.317131,23.559084,64
3,347.092550205901,-2.19213688831977,18.024233 .. 15.605037,0.0026223608 .. 0.0015591898,15.605037,0.0015591898,0.2625042,0.011154525,3.97064,-1.0,-1.0,0.26911718,0.29409084,163.5833,4.2945933,158.96104,4.0634966,1.1034377,1.0123166,0.011719465,0.0,1.4740494 .. 0.31551334,0.00036889763 .. 0.00010674338,0.0002983139 .. 0.0,2.3309622,0.29250583,0.0057010963,1,0.09085343,395.27588,-0.21557358,395.0603,0.26078478 .. 0.32422686,2.10779e-05 .. 5.034007e-06,1,2.4334204,347.089282760688,-2.19067422831961,1.7688129,4.4487863,151.61554,170.70558,33972.55,163.52493,0.30298,5,2,347.092550205901 .. 347.099384487043,-2.19213688831977 .. -2.19344409486259,113660691 .. 113660879,163.5833 .. 163.23244,0.2932724 .. 0.0,1.0 .. 1.0,0.961471 .. 5.574278e-06,0.9608368 .. 5.570601e-06,0.001909672 .. 1.5593596e-05,-26.678312,0.036619306 .. 0.9999788,0.99931395 .. 0.008374583,12.810082,133.82005,0.2932724,0.0022586877,0.1667041,22.081652,22.081652,23.559084,35
4,79.1556849306587,-54.5004588211563,18.133629 .. 15.721151,0.0022215266 .. 0.0012614628,15.721151,0.0012614628,0.27702516,0.011471387,2.4610074,-1.0,-1.0,0.24699661,0.29913905,207.24304,7.181398,181.34009,5.792674,1.1569004,1.0577339,0.05819541,0.0,1.4695177 .. 0.2913362,0.00016469629 .. 5.652495e-05,0.00013060181 .. 0.0,0.3507327,0.29875508,0.005802202,2,0.026094371,392.36365,-0.53134996,391.8323,0.26650682 .. 0.33100334,2.5085239e-05 .. 5.871657e-06,1,2.7067654,79.1563824042035,-54.5140756023846,1.1603314,3.4035742,-70.49273,204.70142,332.49332,207.28362,0.2338536,5,3,79.1556849306587 .. 79.1512336811227,-54.5004588211563 .. -54.5074969897033,416771661 .. 416773652,207.24304 .. 210.72118,0.29971552 .. 0.0,1.0 .. 1.0,0.8949492 .. 3.242092e-07,0.8846991 .. 3.2049593e-07,0.038339663 .. 0.000764328,-15.530748,0.

Below we will create a boolean array using `np.isin()` method. The condition we use is whether a value in column `MEM_MATCH_ID` from `catalog_matcha` is in the column `MEM_MATCH_ID` from `catalog_gold`. 

Once we have the boolean array, we create a column called `isin_Vol_Lim` and give it the array as it's column values. Notice we use `astype(np.int64)`, this is used to convert the boolean values to numerical values of '0 and 1'.

In [7]:
bucket_2 = np.isin(catalog_matcha['MEM_MATCH_ID'], catalog_gold['MEM_MATCH_ID'])

catalog_matcha["isin_Vol_Lim"] = bucket_2.astype(np.int64)

catalog_matcha[0:3]

Name,MEM_MATCH_ID,Analyzed,Detected,redMaPPer_ra,redMaPPer_dec,Redshift,Obsids,lambda,lambda_err_low,lambda_err_high,band_lumin_upper_lim_undetected,band_lumin_upper_lim_detected,lumin_no_tx,lumin_no_tx_err_low,lumin_no_tx_err_high,x_ray_peak_ra,x_ray_peak_dec,500_kiloparsecs_ra,500_kiloparsecs_dec,500_kiloparsecs_temperature,500_kiloparsecs_temperature_err_low,500_kiloparsecs_temperature_err_high,500_kiloparsecs_band_lumin,500_kiloparsecs_band_lumin_err_low,500_kiloparsecs_band_lumin_err_high,500_kiloparsecs_bolo_lumin,500_kiloparsecs_bolo_lumin_err_low,500_kiloparsecs_bolo_lumin_err_high,500_kiloparsecs_radius,500_kiloparsecs_radius_err_low,500_kiloparsecs_radius_err_high,500_kiloparsecs_SNR,500_kiloparsecs_SNR_err_low,500_kiloparsecs_SNR_err_high,r2500_ra,r2500_dec,r2500_temperature,r2500_temperature_err_low,r2500_temperature_err_high,r2500_band_lumin,r2500_band_lumin_err_low,r2500_band_lumin_err_high,r2500_bolo_lumin,r2500_bolo_lumin_err_low,r2500_bolo_lumin_err_high,r2500_radius,r2500_radius_err_low,r2500_radius_err_high,r2500_SNR,r2500_SNR_err_low,r2500_SNR_err_high,r500_ra,r500_dec,r500_temperature,r500_temperature_err_low,r500_temperature_err_high,r500_band_lumin,r500_band_lumin_err_low,r500_band_lumin_err_high,r500_bolo_lumin,r500_bolo_lumin_err_low,r500_bolo_lumin_err_high,r500_radius,r500_radius_err_low,r500_radius_err_high,r500_SNR,r500_SNR_err_low,r500_SNR_err_high,r500_core_cropped_ra,r500_core_cropped_dec,r500_core_cropped_temperature,r500_core_cropped_temperature_err_low,r500_core_cropped_temperature_err_high,r500_core_cropped_band_lumin,r500_core_cropped_band_lumin_err_low,r500_core_cropped_band_lumin_err_high,r500_core_cropped_bolo_lumin,r500_core_cropped_bolo_lumin_err_low,r500_core_cropped_bolo_lumin_err_high,r500_core_cropped_radius,r500_core_cropped_radius_err_low,r500_core_cropped_radius_err_high,r500_core_cropped_SNR,r500_core_cropped_SNR_err_low,r500_core_cropped_SNR_err_high,peak_fix,masked,bad_mode,quick look bad,obsid_bad,merger,Serendipitous,edge_centroid,edge_peak,edge_r2500,edge_r500,edge_bkgd,overlap_r2500,overlap_r500,overlap_bkgd,notes,Group 1 Check,redmapper wrong,right RA,right DEC,right in top 5,right not member,ambiguous,Chandra Notes,Tx_in_xmm,isin_Vol_Lim
bytes15,int64,bool,bool,float64,float64,float64,bytes96,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,bytes4,int64,int64,bytes4,bytes26,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,bytes106,bytes14,int64,float64,float64,int64,int64,int64,bytes145,int32,int64
catalogue_1,2,True,True,83.231901970306,-37.0266894362176,0.2833377420902252,15112,199.432,5.87942,5.87942,nan,nan,4.5843e+44,4.815e+42,4.8632e+42,83.2325,-37.02649,83.23211,-37.02644,11.344,0.68483,0.68878,4.3399e+44,6.51e+42,6.5277e+42,1.7469e+45,2.866e+43,2.9201e+43,500.0,0.0,0.0,94.277,1.087,1.087,83.23188,-37.02643,11.166,0.63832,0.6424,4.944e+44,6.9038e+42,6.8134e+42,1.9782e+45,3.086e+43,3.0796e+43,672.57,23.048,22.571,101.61,1.093,1.093,83.23226,-37.02672,10.934,0.85751,0.85747,5.3675e+44,8.55e+42,8.5518e+42,2.1305e+45,4.5892e+43,4.6121e+43,1511.0,49.857,48.953,79.355,1.039,1.039,83.23226,-37.02672,12.746,1.8843,2.0198,2.7238e+44,6.0129e+42,8.2416e+42,1.1614e+45,6.8477e+43,2.9554e+43,1511.0,49.857,48.953,46.384,1.0189,1.0189,N/A,0,0,N/A,N/A,0,0,0,0,0,0,0,0,0,0,N/A,yes,0,9999.0,9999.0,0,0,0,N/A,1,1
catalogue_2,3,True,True,347.092550205901,-2.19213688831977,0.29250583052635193,"4962, 9372",163.58

Now we save our work using `catalog.write("filepath/filename.fits", format='fits')`

In [8]:
catalog_matcha.write("./filename.fits", format='fits')